In [1]:
from paddleocr import PaddleOCR
import pandas as pd
import numpy as np
import unicodedata
import json
import re
import cv2
import os
import re

pd.set_option('future.no_silent_downcasting', True)

print("Librerias Cargadas")

Librerias Cargadas


In [3]:
#ocr = PaddleOCR(
#    use_doc_orientation_classify=False, 
 #   use_doc_unwarping=False, 
 #   use_textline_orientation=False) # text detection + text recognition

# text image preprocessing + text detection + textline orientation classification + text recognition
ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False, lang="es")

# ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False) # text detection + textline orientation classification + text recognition
# ocr = PaddleOCR(
#     text_detection_model_name="PP-OCRv5_mobile_det",
#     text_recognition_model_name="PP-OCRv5_mobile_rec",
#     use_doc_orientation_classify=False,
#     use_doc_unwarping=False,
#     use_textline_orientation=False) # Switch to PP-OCRv5_mobile models

AttributeError: module 'paddle' has no attribute 'device'

In [ ]:
image_dir = "./imagenes"  # ruta a tu carpeta con imágenes
image_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# Filtramos solo imágenes
image_files = sorted(
    [os.path.join(image_dir, f) 
     for f in os.listdir(image_dir) 
     if os.path.splitext(f)[1].lower() in image_extensions],
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
)

print(str(len(image_files))+" Imagenes Cargadas")

# Tal vez mejorar, si se divide con lineas, o limites

In [ ]:
def ocr_to_multidimensional_sections(result, line_gap=6.5, cortes=None):
    texts = result["rec_texts"]
    boxes = result["rec_boxes"]

    # Extraer coordenadas y asociar texto
    data = []
    for text, box in zip(texts, boxes):
        x_min, y_min = box[0], box[1]  # esquina superior izq
        text = clean_text_simple(text)
        data.append({"text": text, "x": x_min, "y": y_min})

    # Ordenar primero por Y (vertical) y luego por X (horizontal)
    data = sorted(data, key=lambda d: (d["y"], d["x"]))

    # Agrupamos por líneas (Y)
    ordered_lines = []
    current_line = []
    last_y = None

    for item in data:
        if last_y is None or abs(item["y"] - last_y) <= line_gap:
            current_line.append(item)
        else:
            ordered_lines.append(current_line)
            current_line = [item]
        last_y = item["y"]

    if current_line:
        ordered_lines.append(current_line)

    # Si no se pasan cortes, devolvemos normal
    if not cortes:
        max_len = max(len(line) for line in ordered_lines)
        ordered_filled = [
            [i["text"] for i in line] + [""] * (max_len - len(line))
            for line in ordered_lines
        ]
        return pd.DataFrame(ordered_filled)

    # --- Usar cortes en X para dividir en secciones ---
    cortes = sorted(cortes)  # [178, 286, 373] por ejemplo
    n_sections = len(cortes) + 1

    all_rows = []
    for line in ordered_lines:
        row = [""] * n_sections
        for item in line:
            x = item["x"]
            text = item["text"]

            # Buscar en qué sección cae
            section_idx = 0
            for c in cortes:
                if x > c:
                    section_idx += 1
                else:
                    break
            row[section_idx] += (" " + text if row[section_idx] else text)

        all_rows.append(row)

    df = pd.DataFrame(all_rows)
    return df

In [ ]:
def clean_text_simple(text):
    if not isinstance(text, str):
        return text
     # lista por defecto a eliminar (puedes ampliarla)
    default_remove = ["continua...", "continua", "ejemplo", "borrar"]
    removes = default_remove

    s = text.strip()

    # normalizar y eliminar diacríticos (á -> a, ñ -> n, etc.)
    s = unicodedata.normalize("NFKD", s)
    s = s.encode("ascii", "ignore").decode("ascii")

    # eliminar apostrofes rectos y curvos y guion bajo
    s = re.sub(r"[\'\u2019_]", "", s)

    # construir patrón para eliminar palabras/frases como palabras completas
    # normalizamos las frases a ascii (ya lo hicimos arriba) y escapamos
    # usamos boundaries \b para evitar quitar partes dentro de otras palabras
    escaped = [re.escape(r) for r in removes if r]
    if escaped:
        pattern = r"\b(?:" + "|".join(escaped) + r")\b"
        s = re.sub(pattern, " ", s, flags=re.IGNORECASE)

    # remover múltiples espacios y strip final
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [ ]:
def crear_imagenes_con_lineas(promedios):
    output_dir = "./imagenes_con_lineas"
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in image_files:
        img = cv2.imread(img_path)
        if img is None:
            print(f"❌ No se pudo leer: {img_path}")
            continue
    
        # Obtener dimensiones
        height, width, _ = img.shape
    
        # Dibujar cada línea
        for x in promedios:
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.line(img, (int(x), 0), (int(x), height), color, thickness)
    
        # Guardar nueva imagen
        output_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, img)
        print(f"✅ Guardada: {output_path}")


In [ ]:
all_texts = {}

promedios = [120,680,800]

resp = input("¿Deseas continuar? (s/n): ").strip().lower()
if resp in ("s", "si", "y", "yes"):
    print("Usuario dijo SÍ")
    crear_imagenes_con_lineas(promedios)
else:
    print("Usuario dijo NO")

for img_path in image_files[:10]:
    print(img_path)
    res = ocr.predict(img_path)

    df_to_add = ocr_to_multidimensional_sections(res[0], line_gap=6.5, cortes=promedios)
    #print(df_to_add)
    all_texts[os.path.basename(img_path)] = df_to_add
#print(all_texts)

# Guardar todo en un solo Excel

In [ ]:
excel_path = "ocr_resultado_completo.xlsx"

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(all_texts.values(), ignore_index=True)

# Eliminar filas donde TODAS las columnas estén vacías o con espacios
final_df = final_df.replace(r'^\s*$', np.nan, regex=True)  # convierte espacios en NaN
final_df = final_df.dropna(how='all')  # elimina filas donde todo es NaN

resp_save_original = input("¿Deseas continuar? (s/n): ").strip().lower()
if resp_save_original in ("s", "si", "y", "yes"):
    print("Usuario dijo SÍ")
    final_df.to_excel("ocr_origen.xlsx", index=False, header=False)
else:
    print("Usuario dijo NO")

#------------------- Rellenar con Año si esta Presente --------------------

def separar_anio_y_resto(texto):
    if not isinstance(texto, str):
        return np.nan, texto
    partes = texto.strip().split(maxsplit=1)
    if len(partes) == 0:
        return np.nan, texto
    primer = partes[0]
    resto = partes[1] if len(partes) > 1 else ""
    if primer.isdigit() and 1900 <= int(primer) <= 2099:
        return primer, resto
    return np.nan, texto

# Copiamos el DataFrame original
df = final_df.copy()

# Detectamos año y resto
anio_resto = df[1].apply(lambda x: pd.Series(separar_anio_y_resto(x)))
df["año"] = anio_resto[0]
df["texto"] = anio_resto[1]

# Propagamos el año hacia abajo
df["año"] = df["año"].ffill()

# Detectamos modelo y versión
modelos = []
versiones = []
modelo_actual = None

for texto in df["texto"]:
    if isinstance(texto, str) and texto.strip() != "":
        # Si el texto es una palabra corta (como "MDX", "RDX", etc.)
        # o proviene de una fila con año, lo tomamos como modelo
        if len(texto.split()) <= 3:  # criterio: texto corto → modelo
            modelo_actual = texto.strip()
            modelos.append(modelo_actual)
            versiones.append(np.nan)
        else:
            # Si ya hay un modelo actual, esta fila es una versión de ese modelo
            modelos.append(modelo_actual)
            versiones.append(texto.strip())
    else:
        modelos.append(modelo_actual)
        versiones.append(np.nan)

df["modelo"] = modelos
df["version"] = versiones

# 4️⃣ Reordenamos columnas
columnas_ordenadas = ["año", "modelo", "version"] + [
    col for col in df.columns if col not in [0, 1, "texto", "modelo", "version", "año"]
]

excel_df = df[columnas_ordenadas]

# Mostrar resultado
display(excel_df)

## Guardar en una sola hoja
excel_df.to_excel(excel_path, index=False, header=False)

print(f"¡Listo! Todo guardado en {excel_path}")